In [175]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/train-1/train-2.npy
/kaggle/input/train-1/train-4.npy
/kaggle/input/train-1/train-3.npy
/kaggle/input/train-1/train-1.npy
/kaggle/input/datasets-chinese-recognition/random_labels.csv
/kaggle/input/datasets-chinese-recognition/train-2.npy/train-2.npy
/kaggle/input/datasets-chinese-recognition/test.npy/test.npy
/kaggle/input/datasets-chinese-recognition/train-4.npy/train-4.npy
/kaggle/input/datasets-chinese-recognition/train-3.npy/train-3.npy
/kaggle/input/datasets-chinese-recognition/train-1.npy/train-1.npy


In [176]:
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
print(keras.__version__)

2.1.0-rc0
2.3.1


In [180]:
def reset_tf_session():
    curr_session = tf.compat.v1.get_default_session()
    if curr_session is not None:
        curr_session.close()
    K.clear_session()
    config = tf.compat.v1.ConfigProto
    config.gpu_options.allow_growth = True
    s = tf.compat.v1.InteractiveSession(config=config)
    tf.compat.v1.keras.backend.set_session(s)
    return s

In [181]:
data_train = np.load("/kaggle/input/train-1/train-1.npy", allow_pickle=True)
for i in range(2, 5):
    t = np.load(f"/kaggle/input/train-1/train-{i}.npy", allow_pickle=True)
    data_train = np.concatenate([data_train, t])
data_test = np.load('/kaggle/input/datasets-chinese-recognition/test.npy/test.npy', allow_pickle=True)

In [182]:
t = np.unique([v[1] for v in data_train])
to_char = {id:char for id, char in enumerate(t)}
to_id = {char:id for id, char in enumerate(t)}


In [183]:
val_size = 0.3

def train_gen():
    for img, label in data_train[int((val_size * len(data_train))):]:
        img = img[:, :, np.newaxis] # [batch, w, h, channels]
        label = to_id[label]
        yield img, label
        
def preprocess_train(img, label):
    img = tf.image.resize(img, (80, 80))
    img = tf.cast(img, tf.float32)
    img = img / 255 - 0.5
    label = tf.one_hot(label, 1000)
    return img, label


def val_gen():
    for img, label in data_train[:int(val_size * len(data_train))]:
        img = img[:, :, np.newaxis]
        label = to_id[label]
        yield img, label
        
def preprocess_val(img, label):
    img = tf.image.resize(img, (80, 80))
    img = tf.cast(img, tf.float32)
    img = img / 255 - 0.5
    label = tf.one_hot(label, 1000)
    return img, label

def test_gen():
    for img in data_test:
        img = img[:, :, np.newaxis] # [batch, w, h, channels]
        yield img

def preprocess_test(img):
    img = tf.image.resize(img, (80, 80))
    img = tf.cast(img, tf.float32)
    img = img / 255 - 0.5
    return img

In [184]:
ds_train = tf.data.Dataset.from_generator(train_gen,
                                          output_types=(tf.uint8, tf.int32),
                                          output_shapes=((None,None,1), ()))
ds_train = ds_train\
.map(preprocess_train, num_parallel_calls=-1)\
.shuffle(1024)\
.repeat()\
.prefetch(-1)\
.batch(256)

ds_val = tf.data.Dataset.from_generator(val_gen,
                                          output_types=(tf.uint8, tf.int32),
                                          output_shapes=((None,None,1), ()))

ds_val = ds_val\
.map(preprocess_val, num_parallel_calls=-1)\
.shuffle(1024)\
.repeat()\
.prefetch(-1)\
.batch(256)


In [203]:
ds_test = tf.data.Dataset.from_generator(test_gen,
                                          output_types=tf.uint8,
                                          output_shapes=((None,None,1)))

ds_test = ds_test\
.map(preprocess_test, num_parallel_calls=-1)\
.prefetch(-1)\
.batch(256)

In [186]:
def make_model(): 
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), input_shape=(80, 80, 1), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))  
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))  
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.Conv2D(filters=64, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))  
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same')) 
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.Conv2D(filters=128, padding='same', kernel_size=(3,3), kernel_initializer='he_uniform'))  
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='same')) 
    model.add(tf.keras.layers.BatchNormalization()) 
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.LeakyReLU(0.1))
    model.add(tf.keras.layers.BatchNormalization())    
    model.add(tf.keras.layers.Dense(1000, kernel_initializer='he_uniform'))             
    model.add(tf.keras.layers.Activation("softmax"))
    
    return model

In [187]:
def fit_model(INIT_LR=5e-3, BATCH_SIZE=256, epoch=20):
 
    def lr_scheduler(epoch):
        return INIT_LR * 0.9 ** epoch  

    model = make_model()
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adamax(lr=INIT_LR),
        metrics=['accuracy']
    )
  
    class LrHistory(tf.keras.callbacks.Callback):

        def on_epoch_begin(self, epoch, logs={}):
            print("Learning_rate:", tf.keras.backend.get_value(model.optimizer.lr))

            
    
    history = model.fit(ds_train,
        epochs=epoch,
        callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler),
                   LrHistory(),],
        validation_data=ds_val,                   
        shuffle=True,
        verbose=1,
        initial_epoch=0, validation_steps=int(val_size*data_train.shape[0]/(BATCH_SIZE)), steps_per_epoch= int((1-val_size)*data_train.shape[0]/(BATCH_SIZE)))

    return history

In [188]:
fit_model()

Train for 910 steps, validate for 390 steps
Learning_rate: 0.005
Epoch 1/20
910/910 [==============================] - 259s 285ms/step - loss: 1.1457 - accuracy: 0.7657 - val_loss: 0.3734 - val_accuracy: 0.8963
Learning_rate: 0.0045
Epoch 2/20
910/910 [==============================] - 194s 213ms/step - loss: 0.1674 - accuracy: 0.9537 - val_loss: 0.2621 - val_accuracy: 0.9252
Learning_rate: 0.00405
Epoch 3/20
910/910 [==============================] - 192s 211ms/step - loss: 0.0995 - accuracy: 0.9718 - val_loss: 0.1697 - val_accuracy: 0.9530
Learning_rate: 0.003645
Epoch 4/20
910/910 [==============================] - 194s 213ms/step - loss: 0.0587 - accuracy: 0.9833 - val_loss: 0.1312 - val_accuracy: 0.9634
Learning_rate: 0.0032805
Epoch 5/20
910/910 [==============================] - 192s 211ms/step - loss: 0.0357 - accuracy: 0.9900 - val_loss: 0.0843 - val_accuracy: 0.9784
Learning_rate: 0.00295245
Epoch 6/20
910/910 [==============================] - 193s 212ms/step - loss: 0.0203 

In [201]:
model = make_model()
model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adamax(lr=5e-3),
        metrics=['accuracy']
    )
model.fit(ds_train,
        epochs=20,
        validation_data=ds_val,                   
        shuffle=True,
        verbose=1,
        initial_epoch=0, validation_steps=int(val_size*data_train.shape[0]/(256)), steps_per_epoch= int((1-val_size)*data_train.shape[0]/(256)))
#sub = model.predict(ds_test) - это то, из за чего ошибка выдалась, перепрогружать не буду, очень долго
#sub 

Train for 910 steps, validate for 390 steps
Epoch 1/20
910/910 [==============================] - 191s 210ms/step - loss: 1.1299 - accuracy: 0.7688 - val_loss: 2.9846 - val_accuracy: 0.4273
Epoch 2/20
910/910 [==============================] - 188s 206ms/step - loss: 0.1824 - accuracy: 0.9489 - val_loss: 0.2665 - val_accuracy: 0.9264
Epoch 3/20
910/910 [==============================] - 190s 208ms/step - loss: 0.1196 - accuracy: 0.9655 - val_loss: 0.2402 - val_accuracy: 0.9326
Epoch 4/20
910/910 [==============================] - 187s 206ms/step - loss: 0.0832 - accuracy: 0.9754 - val_loss: 0.1786 - val_accuracy: 0.9499
Epoch 5/20
910/910 [==============================] - 186s 204ms/step - loss: 0.0593 - accuracy: 0.9824 - val_loss: 0.1230 - val_accuracy: 0.9670
Epoch 6/20
910/910 [==============================] - 187s 206ms/step - loss: 0.0425 - accuracy: 0.9871 - val_loss: 0.1308 - val_accuracy: 0.9655
Epoch 7/20
910/910 [==============================] - 186s 205ms/step - loss: 0.

ValueError: in converted code:

    /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_v2.py:671 map_fn
        batch_size=None)
    /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py:2371 _standardize_tensors
        exception_prefix='input')
    /opt/conda/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_utils.py:573 standardize_input_data
        'with shape ' + str(data_shape))

    ValueError: Error when checking input: expected conv2d_193_input to have 4 dimensions, but got array with shape (None, None, 80, 80, 1)


Да, тут ошибка выдалась, но к счастью сама модель обучилась, а проблему я устранила и ниже всё норм. Оно грузит больше часа, пожалейте...... не стала перезагружать

In [231]:
sub = model.predict(ds_test)

In [232]:
sub = sub.argmax(1)

In [228]:
sub

array([802, 846, 905, ..., 948, 263,  22])

In [229]:
sub = [to_char[i] for i in sub]
csv = pd.read_csv('/kaggle/input/datasets-chinese-recognition/random_labels.csv')
csv['Category'] = sub
csv.to_csv("submission.csv", index=False)

In [230]:

csv.head()

,Id,Category
0,1,63955
1,2,64432
2,3,64709
3,4,64177
4,5,61881


Это было жостка. Если я что-то случайно удалила что нужно было или оставила какую-то ненужную штуку, то простите, нет сил и времени в порядок тетрадь привести........... на кегеле сабмитнула, всё ок